In [1]:
import argparse
import os
import logging
import numpy as np
import tensorflow as tf

from dataset import KinQueryDataset, preprocess

In [2]:
MARK_PAD = "<PAD>"
MARK_UNK = "<UNK>"

MARKS = [MARK_PAD, MARK_UNK]
ID_PAD = 0
ID_UNK = 1

In [3]:
def _batch_loader(iterable, n=1):
    """
    데이터를 배치 사이즈만큼 잘라서 보내주는 함수입니다. PyTorch의 DataLoader와 같은 역할을 합니다

    :param iterable: 데이터 list, 혹은 다른 포맷
    :param n: 배치 사이즈
    :return:
    """
    length = len(iterable)
    for n_idx in range(0, length, n):
        yield iterable[n_idx:min(n_idx + n, length)]


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
def load_dict(dict_path, max_vocab=None):
    logging.info("Try load dict from {}.".format(dict_path))
    try:
        dict_file = open(dict_path)
        dict_data = dict_file.readlines()
        dict_file.close()
    except:
        logging.info(
            "Load dict {dict} failed, create later.".format(dict=dict_path))
        return None

    dict_data = list(map(lambda x: x.split(), dict_data))
    if max_vocab:
        dict_data = list(filter(lambda x: int(x[0]) < max_vocab, dict_data))
    tok2id = dict(map(lambda x: (x[1], int(x[0])), dict_data))
    id2tok = dict(map(lambda x: (int(x[0]), x[1]), dict_data))
    logging.info(
        "Load dict {} with {} words.".format(dict_path, len(tok2id)))
    return (tok2id, id2tok)

def create_dict(dict_path, corpus, max_vocab=None):
    logging.info("Create dict {}.".format(dict_path))
    counter = {}
    for line in corpus:
        for word in line:
            try:
                counter[word] += 1
            except:
                counter[word] = 1

    for mark_t in MARKS:
        if mark_t in counter:
            del counter[mark_t]
            logging.warning("{} appears in corpus.".format(mark_t))

    counter = list(counter.items())
    counter.sort(key=lambda x: -x[1])
    words = list(map(lambda x: x[0], counter))
    words = [MARK_PAD, MARK_UNK] + words
    if max_vocab:
        words = words[:max_vocab]

    tok2id = dict()
    id2tok = dict()
    with open(dict_path, 'w') as dict_file:
        for idx, tok in enumerate(words):
            print(idx, tok, file=dict_file)
            tok2id[tok] = idx
            id2tok[idx] = tok

    logging.info(
        "Create dict {} with {} words.".format(dict_path, len(words)))
    return (tok2id, id2tok)

def corpus_map2id(data, tok2id):
    ret = []
    unk = 0
    tot = 0
    for doc in data:
        tmp = []
        for word in doc:
            tot += 1
            try:
                tmp.append(tok2id[word])
            except:
                tmp.append(ID_UNK)
                unk += 1
        ret.append(tmp)
    return ret, (tot - unk) / tot


def sen_map2tok(sen, id2tok):
    return list(map(lambda x: id2tok[x], sen))


def load_data(data,
              doc_dict_path,
              max_doc_vocab=None):
    with open(data,'rt', encoding="utf-8") as docfile:
        docs = docfile.readlines()

    docs = list(map(lambda x: x.split(), data))

    doc_dict = load_dict(doc_dict_path, max_doc_vocab)
    if doc_dict is None:
        doc_dict = create_dict(doc_dict_path, docs, max_doc_vocab)

    docid, cover = corpus_map2id(docs, doc_dict[0])
    print("Doc dict covers %.2f%% words."%(cover*100))

    return docid, doc_dict

def load_test_data(doc, doc_dict):
    docs = corpus_preprocess(doc)

    print("Load %d testing documents."%(len(docs)))
    docs = list(map(lambda x: x.split(), docs))

    docid, cover = corpus_map2id(docs, doc_dict[0])
    print("Doc dict covers %.2f words."%(cover*100))

    return docid

def corpus_preprocess(corpus):
    import re
    ret = []
    for line in corpus:
        x = re.sub('\\d', '#', line)
        ret.append(x)
    return ret


def sen_postprocess(sen):
    return sen

In [5]:
class KinQueryDataset:
    def __init__(self, dataset_path: str, max_length: int, vocab_size: int):
        # 데이터, 레이블 각각의 경로
        queries_path = os.path.join(dataset_path, 'train', 'train_data')
        labels_path = os.path.join(dataset_path, 'train', 'train_label')

        # 지식인 데이터를 읽고 preprocess까지 진행합니다
        dict_name = str(vocab_size)+"dict"
        dicId , dic = load_data(queries_path, dict_name , vocab_size)
        self.queries = preprocess(dicId, dic, max_length)
        # 지식인 레이블을 읽고 preprocess까지 진행합니다.
        
        with open(labels_path) as f:
            self.labels = np.array([[np.float32(x)] for x in f.readlines()])

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        return self.queries[idx], self.labels[idx]

In [6]:
def preprocess(data: list, dic: list, max_length: int):
    vectorized_data = data
    zero_padding = np.zeros((len(data), max_length), dtype=np.int32)
    #print(data)
    for idx, seq in enumerate(vectorized_data):
        length = len(seq)
        #print(idx, seq)
        if length >= max_length:
            length = max_length
            zero_padding[idx, :length] = np.array(seq)[:length]
        elif(length == 0):
            zero_padding[idx,] = zero_padding[idx,]
        else:
            zero_padding[idx,] = np.append(zero_padding[idx,:-length], np.array(seq))
    return zero_padding

In [7]:
#DATASET_PATH = '../sample_data/movie_review/'
#queries_path = os.path.join(DATASET_PATH, 'train', 'train_data')
#with open(queries_path, 'rt', encoding='utf8') as f:
#            print(f.readlines())
#dataset = KinQueryDataset(DATASET_PATH, strmaxlen)

In [8]:
def label_one_hot(labels):
    one_hot = (np.arange(output_size) == labels[:]).astype(np.int32)
    return one_hot
def catFromOut(output):
    idx = []
    for i in output:
        ids = np.argmax(i, axis = 0)
        idx.append(ids)
    idx = np.array(idx)
    return idx

In [9]:
# User options
batch = 1
epochs = 100

embedding = 30
strmaxlen = 50
DATASET_PATH = '../sample_data/movie_review/'

# 모델의 specification
input_size = embedding*strmaxlen
output_size = 1
hidden_layer_size = 200
learning_rate = 0.01
dict_size = 424

x = tf.placeholder(tf.int32, [None, strmaxlen])
y_ = tf.placeholder(tf.float32, [None, output_size])
# 임베딩
char_embedding = tf.get_variable('char_embedding', [dict_size, embedding])
embedded = tf.nn.embedding_lookup(char_embedding, x)

# 첫 번째 레이어
cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer_size)
#cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.9)
#cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer_size)

#multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell, cell2])

output, state = tf.nn.dynamic_rnn(cell, embedded, dtype=tf.float32)
output = tf.transpose(output, [1, 0, 2])
output = output[-1]
print("output shaep",output.shape)

# 두 번째 (아웃풋) 레이어
second_layer_weight = weight_variable([hidden_layer_size, output_size])
second_layer_bias = bias_variable([output_size])
output = tf.matmul(output, second_layer_weight) + second_layer_bias
#output_sigmoid = tf.sigmoid(output)
print("output shape",output.shape)
print("lables shaep",y_.shape)
# loss와 optimizer
#global_step = tf.Variable(0)
foutput = tf.nn.log_softmax(output)
cost = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_, predictions=output))
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y_))
#learning_rate= tf.train.exponential_decay(learning_rate, global_step, 10000, 0.75)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cost)


sess = tf.InteractiveSession()
tf.global_variables_initializer().run()



output shaep (?, 200)
output shape (?, 1)
lables shaep (?, 1)


In [12]:
import time
import math
from sklearn.metrics import mean_squared_error
all_losses=[]
current_loss=0
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()
all_losses=[]
dataset = KinQueryDataset(DATASET_PATH, strmaxlen, 3000)
dataset_len = len(dataset)
one_batch_size = dataset_len//batch
if dataset_len % batch != 0:
    one_batch_size += 1
# epoch마다 학습을 수행합니다.
for epoch in range(epochs):
    avg_loss = 0.0
    for i, (data, labels) in enumerate(_batch_loader(dataset, batch)):
        _, loss = sess.run([train_step, cost],
                           feed_dict={x: data, y_: labels})
        print('Batch : ', i + 1, '/', one_batch_size,
              ', MSE in this minibatch: ', float(loss))
        avg_loss += float(loss)
    all_losses.append(avg_loss/one_batch_size)
    print('epoch:', epoch, ' train_loss:', float(avg_loss/one_batch_size),timeSince(start))

Doc dict covers 100.00% words.
Batch :  1 / 44 , MSE in this minibatch:  2.1008763313293457
Batch :  2 / 44 , MSE in this minibatch:  0.40363234281539917
Batch :  3 / 44 , MSE in this minibatch:  0.4753429889678955
Batch :  4 / 44 , MSE in this minibatch:  2.0045006275177
Batch :  5 / 44 , MSE in this minibatch:  0.5949350595474243
Batch :  6 / 44 , MSE in this minibatch:  0.003333711763843894
Batch :  7 / 44 , MSE in this minibatch:  1.2240713834762573
Batch :  8 / 44 , MSE in this minibatch:  0.8780142068862915
Batch :  9 / 44 , MSE in this minibatch:  1.6380404233932495
Batch :  10 / 44 , MSE in this minibatch:  1.1500113010406494
Batch :  11 / 44 , MSE in this minibatch:  0.144703209400177
Batch :  12 / 44 , MSE in this minibatch:  1.0284956693649292
Batch :  13 / 44 , MSE in this minibatch:  0.27028346061706543
Batch :  14 / 44 , MSE in this minibatch:  0.6616944670677185
Batch :  15 / 44 , MSE in this minibatch:  0.4481809139251709
Batch :  16 / 44 , MSE in this minibatch:  0.017

Batch :  1 / 44 , MSE in this minibatch:  2.412386655807495
Batch :  2 / 44 , MSE in this minibatch:  0.7965688705444336
Batch :  3 / 44 , MSE in this minibatch:  2.37135910987854
Batch :  4 / 44 , MSE in this minibatch:  0.2368760108947754
Batch :  5 / 44 , MSE in this minibatch:  4.38528299331665
Batch :  6 / 44 , MSE in this minibatch:  2.3727455139160156
Batch :  7 / 44 , MSE in this minibatch:  0.00016752818191889673
Batch :  8 / 44 , MSE in this minibatch:  0.00038895668694749475
Batch :  9 / 44 , MSE in this minibatch:  7.667553901672363
Batch :  10 / 44 , MSE in this minibatch:  4.780038356781006
Batch :  11 / 44 , MSE in this minibatch:  1.5491628646850586
Batch :  12 / 44 , MSE in this minibatch:  2.77736234664917
Batch :  13 / 44 , MSE in this minibatch:  0.9509195685386658
Batch :  14 / 44 , MSE in this minibatch:  0.9076548218727112
Batch :  15 / 44 , MSE in this minibatch:  0.3246387839317322
Batch :  16 / 44 , MSE in this minibatch:  0.31925255060195923
Batch :  17 / 44 

Batch :  5 / 44 , MSE in this minibatch:  4.246535301208496
Batch :  6 / 44 , MSE in this minibatch:  2.0395612716674805
Batch :  7 / 44 , MSE in this minibatch:  0.013308869674801826
Batch :  8 / 44 , MSE in this minibatch:  0.010073453187942505
Batch :  9 / 44 , MSE in this minibatch:  6.7137064933776855
Batch :  10 / 44 , MSE in this minibatch:  4.010688304901123
Batch :  11 / 44 , MSE in this minibatch:  1.028135895729065
Batch :  12 / 44 , MSE in this minibatch:  2.2180564403533936
Batch :  13 / 44 , MSE in this minibatch:  0.4446801245212555
Batch :  14 / 44 , MSE in this minibatch:  0.6905108094215393
Batch :  15 / 44 , MSE in this minibatch:  0.23709049820899963
Batch :  16 / 44 , MSE in this minibatch:  0.4157439172267914
Batch :  17 / 44 , MSE in this minibatch:  0.02548046223819256
Batch :  18 / 44 , MSE in this minibatch:  0.013176735490560532
Batch :  19 / 44 , MSE in this minibatch:  0.012586651369929314
Batch :  20 / 44 , MSE in this minibatch:  0.09633277356624603
Batch

Batch :  6 / 44 , MSE in this minibatch:  1.9503231048583984
Batch :  7 / 44 , MSE in this minibatch:  0.03688466176390648
Batch :  8 / 44 , MSE in this minibatch:  0.029868431389331818
Batch :  9 / 44 , MSE in this minibatch:  6.4825286865234375
Batch :  10 / 44 , MSE in this minibatch:  3.8216183185577393
Batch :  11 / 44 , MSE in this minibatch:  0.892936110496521
Batch :  12 / 44 , MSE in this minibatch:  2.0758352279663086
Batch :  13 / 44 , MSE in this minibatch:  0.3649831712245941
Batch :  14 / 44 , MSE in this minibatch:  0.6327592730522156
Batch :  15 / 44 , MSE in this minibatch:  0.21253107488155365
Batch :  16 / 44 , MSE in this minibatch:  0.46181175112724304
Batch :  17 / 44 , MSE in this minibatch:  0.023547491058707237
Batch :  18 / 44 , MSE in this minibatch:  0.013481940142810345
Batch :  19 / 44 , MSE in this minibatch:  0.010376828722655773
Batch :  20 / 44 , MSE in this minibatch:  0.08559560030698776
Batch :  21 / 44 , MSE in this minibatch:  0.8748553991317749
B

Batch :  7 / 44 , MSE in this minibatch:  0.012896893545985222
Batch :  8 / 44 , MSE in this minibatch:  0.011810463853180408
Batch :  9 / 44 , MSE in this minibatch:  6.6335296630859375
Batch :  10 / 44 , MSE in this minibatch:  3.992553472518921
Batch :  11 / 44 , MSE in this minibatch:  0.9756030440330505
Batch :  12 / 44 , MSE in this minibatch:  2.2296271324157715
Batch :  13 / 44 , MSE in this minibatch:  0.44993582367897034
Batch :  14 / 44 , MSE in this minibatch:  0.7151718139648438
Batch :  15 / 44 , MSE in this minibatch:  0.25646716356277466
Batch :  16 / 44 , MSE in this minibatch:  0.41707688570022583
Batch :  17 / 44 , MSE in this minibatch:  0.008365321904420853
Batch :  18 / 44 , MSE in this minibatch:  0.021609371528029442
Batch :  19 / 44 , MSE in this minibatch:  0.004540314897894859
Batch :  20 / 44 , MSE in this minibatch:  0.06590739637613297
Batch :  21 / 44 , MSE in this minibatch:  0.80061274766922
Batch :  22 / 44 , MSE in this minibatch:  0.25739142298698425

...

Batch :  2 / 44 , MSE in this minibatch:  1.4322041273117065
Batch :  3 / 44 , MSE in this minibatch:  2.7975869178771973
Batch :  4 / 44 , MSE in this minibatch:  0.08974966406822205
Batch :  5 / 44 , MSE in this minibatch:  4.354040145874023
Batch :  6 / 44 , MSE in this minibatch:  1.7399076223373413
Batch :  7 / 44 , MSE in this minibatch:  0.007699636742472649
Batch :  8 / 44 , MSE in this minibatch:  0.0014850287698209286
Batch :  9 / 44 , MSE in this minibatch:  6.5440473556518555
Batch :  10 / 44 , MSE in this minibatch:  3.7678956985473633
Batch :  11 / 44 , MSE in this minibatch:  0.7254543304443359
Batch :  12 / 44 , MSE in this minibatch:  2.10124135017395
Batch :  13 / 44 , MSE in this minibatch:  0.5879514217376709
Batch :  14 / 44 , MSE in this minibatch:  0.6821014881134033
Batch :  15 / 44 , MSE in this minibatch:  0.24775724112987518
Batch :  16 / 44 , MSE in this minibatch:  0.526183009147644
Batch :  17 / 44 , MSE in this minibatch:  1.4844408724457026e-07
Batch :  

Batch :  2 / 44 , MSE in this minibatch:  1.4236013889312744
Batch :  3 / 44 , MSE in this minibatch:  2.814523935317993
Batch :  4 / 44 , MSE in this minibatch:  0.00014933774946257472
Batch :  5 / 44 , MSE in this minibatch:  4.485182762145996
Batch :  6 / 44 , MSE in this minibatch:  1.858959674835205
Batch :  7 / 44 , MSE in this minibatch:  0.00025887007359415293
Batch :  8 / 44 , MSE in this minibatch:  0.0005787579575553536
Batch :  9 / 44 , MSE in this minibatch:  7.016042232513428
Batch :  10 / 44 , MSE in this minibatch:  4.194104194641113
Batch :  11 / 44 , MSE in this minibatch:  0.9379001259803772
Batch :  12 / 44 , MSE in this minibatch:  2.4335269927978516
Batch :  13 / 44 , MSE in this minibatch:  0.7579505443572998
Batch :  14 / 44 , MSE in this minibatch:  0.84138023853302
Batch :  15 / 44 , MSE in this minibatch:  0.3289598524570465
Batch :  16 / 44 , MSE in this minibatch:  0.43720000982284546
Batch :  17 / 44 , MSE in this minibatch:  0.0008309794357046485
Batch : 

Batch :  6 / 44 , MSE in this minibatch:  1.8529598712921143
Batch :  7 / 44 , MSE in this minibatch:  2.591536394902505e-05
Batch :  8 / 44 , MSE in this minibatch:  0.00019116851035505533
Batch :  9 / 44 , MSE in this minibatch:  7.003493309020996
Batch :  10 / 44 , MSE in this minibatch:  4.179747581481934
Batch :  11 / 44 , MSE in this minibatch:  0.9297055602073669
Batch :  12 / 44 , MSE in this minibatch:  2.421740770339966
Batch :  13 / 44 , MSE in this minibatch:  0.75165194272995
Batch :  14 / 44 , MSE in this minibatch:  0.8353185057640076
Batch :  15 / 44 , MSE in this minibatch:  0.3255479335784912
Batch :  16 / 44 , MSE in this minibatch:  0.4412855803966522
Batch :  17 / 44 , MSE in this minibatch:  0.00017714222485665232
Batch :  18 / 44 , MSE in this minibatch:  0.06705325096845627
Batch :  19 / 44 , MSE in this minibatch:  0.003356326138600707
Batch :  20 / 44 , MSE in this minibatch:  0.02792303077876568
Batch :  21 / 44 , MSE in this minibatch:  0.8090640306472778
Ba

Batch :  8 / 44 , MSE in this minibatch:  4.7226818423951045e-05
Batch :  9 / 44 , MSE in this minibatch:  7.00713586807251
Batch :  10 / 44 , MSE in this minibatch:  4.1757636070251465
Batch :  11 / 44 , MSE in this minibatch:  0.9242203235626221
Batch :  12 / 44 , MSE in this minibatch:  2.4130663871765137
Batch :  13 / 44 , MSE in this minibatch:  0.7451952695846558
Batch :  14 / 44 , MSE in this minibatch:  0.8286135792732239
Batch :  15 / 44 , MSE in this minibatch:  0.32118886709213257
Batch :  16 / 44 , MSE in this minibatch:  0.44808515906333923
Batch :  17 / 44 , MSE in this minibatch:  2.4056542315520346e-05
Batch :  18 / 44 , MSE in this minibatch:  0.06601908802986145
Batch :  19 / 44 , MSE in this minibatch:  0.0037360081914812326
Batch :  20 / 44 , MSE in this minibatch:  0.02801266498863697
Batch :  21 / 44 , MSE in this minibatch:  0.80909663438797
Batch :  22 / 44 , MSE in this minibatch:  0.1997702270746231
Batch :  23 / 44 , MSE in this minibatch:  0.2724285721778869

Batch :  11 / 44 , MSE in this minibatch:  0.9311627149581909
Batch :  12 / 44 , MSE in this minibatch:  2.4234061241149902
Batch :  13 / 44 , MSE in this minibatch:  0.7503197193145752
Batch :  14 / 44 , MSE in this minibatch:  0.833487331867218
Batch :  15 / 44 , MSE in this minibatch:  0.32359957695007324
Batch :  16 / 44 , MSE in this minibatch:  0.4455620050430298
Batch :  17 / 44 , MSE in this minibatch:  2.9861839720979333e-06
Batch :  18 / 44 , MSE in this minibatch:  0.06626778841018677
Batch :  19 / 44 , MSE in this minibatch:  0.0039070844650268555
Batch :  20 / 44 , MSE in this minibatch:  0.028048111125826836
Batch :  21 / 44 , MSE in this minibatch:  0.8093711733818054
Batch :  22 / 44 , MSE in this minibatch:  0.200203537940979
Batch :  23 / 44 , MSE in this minibatch:  0.2730890214443207
Batch :  24 / 44 , MSE in this minibatch:  0.3178423345088959
Batch :  25 / 44 , MSE in this minibatch:  0.4595889151096344
Batch :  26 / 44 , MSE in this minibatch:  0.2984102666378021

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

plt.show()